# this is check the model using no fucntion at all.

In [1]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [2]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P3"))
sys.path.append(str_path_1) 
from Fun_P3 import *
BasicPath = "D:/OneDrive - Imperial College London/SimDataSave/P3R13/" # 
Target  = 'Test_NoFunc/' 
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)

In [3]:
V_max = 4.2;        V_min = 2.5;
exp  = pybamm.Experiment(
    [ (
        f"Discharge at 3 C until {V_min} V", 
        f"Charge at 3 C until {V_max} V", 
        f"Rest for 0.1 h",)    ] * 1 )  

In [4]:
# sol - Dec=5e-10; single vs double 
Model_DFN_sol  = pybamm.lithium_ion.DFN(options={
    "SEI":"solvent-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "single no consume wo refill",
    "electrolyte conductivity": "full"  ,}) 
Model_DD_sol  = pybamm.lithium_ion.DFN(options={
    "SEI":"solvent-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "double no consume wo refill",
    "electrolyte conductivity": "sol full"  ,}) 
Model_All_sol =[ Model_DFN_sol,   Model_DD_sol ]
Str_model_sol =[ 'Model_DFN_sol', 'Model_DD_sol' ]
# Para_All[0]: for single; Para_All[1]: for double
para=pybamm.ParameterValues("Li2023_ECdrag")
Para_All = []
for i in range(0,2):
    para=pybamm.ParameterValues("Li2023_ECdrag")
    para.update({"EC initial concentration in electrolyte [mol.m-3]":3500})
    para.update({"Typical EC concentration [mol.m-3]":3500})
    para.update({'EC diffusivity in SEI [m2.s-1]':5e-20})
    para.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_5E_10})
    para.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':3e-19})
    para.update({"EC Lithium ion cross diffusivity [m2.s-1]":Cross_diffusivity_1p5E_12})
    para.update({"Cation transference number":electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Con})
    Para_All.append(para)
Para_All[0].update({"Measured dLJP_dce":dLJP_One_Specie_dce_Jung2023})
# check 
print(Para_All[0]["Measured dLJP_dce"])
print(Para_All[1]["Measured dLJP_dce"])
Sol_all_sol =[];
for model,para in zip(Model_All_sol,Para_All):
    c_e = model.variables["Electrolyte concentration [mol.m-3]"]
    c_EC= model.variables["EC concentration [mol.m-3]"]
    T = model.variables["Cell temperature [K]"]
    D_e = para["Electrolyte diffusivity [m2.s-1]"]
    D_EC= para["EC diffusivity in electrolyte [m2.s-1]"]
    sigma_e = para["Electrolyte conductivity [S.m-1]"]
    Xi = para["EC transference number"]
    model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
    model.variables["EC diffusivity in electrolyte [m2.s-1]"] = D_EC(c_e,c_EC, T)
    model.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
    model.variables["EC transference number"] = Xi(c_e,c_EC, T)
    model.variables["c(EC) over c(Li+)"] = c_EC / c_e
    t_0plus = para["Cation transference number"]
    model.variables["Cation transference number"] = t_0plus(c_e,c_EC, T)

    var_pts = {
        "x_n": 10,  # negative electrode
        "x_s": 5,  # separator 
        "x_p": 10,  # positive electrode
        "r_n": 30,  # negative particle
        "r_p": 20,  # positive particle
    }

    sim    = pybamm.Simulation(
        model, experiment = exp,
        parameter_values = para,
        solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,)       
    Sol_all_sol.append(sim.solve())

<function dLJP_One_Specie_dce_Jung2023 at 0x000001D07EBD8550>
<function dLJP_Two_Species_dce_Jung2023 at 0x000001D07FDC3E50>


model = Model_DD
c_e = model.variables["Electrolyte concentration [mol.m-3]"]
c_EC= model.variables["EC concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
D_e = para["Electrolyte diffusivity [m2.s-1]"]
D_EC= para["EC diffusivity in electrolyte [m2.s-1]"]
sigma_e = para["Electrolyte conductivity [S.m-1]"]
Xi = para["EC transference number"]
model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
model.variables["EC diffusivity in electrolyte [m2.s-1]"] = D_EC(c_e,c_EC, T)
model.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
model.variables["EC transference number"] = Xi(c_e,c_EC, T)
model.variables["c(EC) over c(Li+)"] = c_EC / c_e
t_0plus = para["Cation transference number"]
model.variables["Cation transference number"] = t_0plus(c_e,c_EC, T)

var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 10,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 20,  # positive particle
}

sim    = pybamm.Simulation(
    model, experiment = exp,
    parameter_values = para,
    solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
    var_pts=var_pts,
    )       
sol  = sim.solve()

In [5]:
# int - Dec=5e-10; single vs double 
Model_DFN_int  = pybamm.lithium_ion.DFN(options={
    "SEI":"interstitial-diffusion limited",            
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "single no consume wo refill",
    "electrolyte conductivity": "full"  ,}) 
Model_DD_int  = pybamm.lithium_ion.DFN(options={
    "SEI":"interstitial-diffusion limited",           
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "double no consume wo refill",
    "electrolyte conductivity": "sol full"  ,}) 
Model_All_int =[ Model_DFN_int,   Model_DD_int ]
Str_model_int =[ 'Model_DFN_int', 'Model_DD_int' ]
# Para_All[0]: for single; Para_All[1]: for double
para=pybamm.ParameterValues("Li2023_ECdrag")
Para_All = []
for i in range(0,2):
    para=pybamm.ParameterValues("Li2023_ECdrag")
    para.update({"EC initial concentration in electrolyte [mol.m-3]":3500})
    para.update({"Typical EC concentration [mol.m-3]":3500})
    para.update({'EC diffusivity in SEI [m2.s-1]':5e-20})
    para.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_5E_10})
    para.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':3e-19})
    para.update({"EC Lithium ion cross diffusivity [m2.s-1]":Cross_diffusivity_1p5E_12})
    para.update({"Cation transference number":electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Con})
    Para_All.append(para)
Para_All[0].update({"Measured dLJP_dce":dLJP_One_Specie_dce_Jung2023})
# check 
print(Para_All[0]["Measured dLJP_dce"])
print(Para_All[1]["Measured dLJP_dce"])
Sol_all_int =[];
for model,para in zip(Model_All_int,Para_All):
    c_e = model.variables["Electrolyte concentration [mol.m-3]"]
    c_EC= model.variables["EC concentration [mol.m-3]"]
    T = model.variables["Cell temperature [K]"]
    D_e = para["Electrolyte diffusivity [m2.s-1]"]
    D_EC= para["EC diffusivity in electrolyte [m2.s-1]"]
    sigma_e = para["Electrolyte conductivity [S.m-1]"]
    Xi = para["EC transference number"]
    model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
    model.variables["EC diffusivity in electrolyte [m2.s-1]"] = D_EC(c_e,c_EC, T)
    model.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
    model.variables["EC transference number"] = Xi(c_e,c_EC, T)
    model.variables["c(EC) over c(Li+)"] = c_EC / c_e
    t_0plus = para["Cation transference number"]
    model.variables["Cation transference number"] = t_0plus(c_e,c_EC, T)

    var_pts = {
        "x_n": 10,  # negative electrode
        "x_s": 5,  # separator 
        "x_p": 10,  # positive electrode
        "r_n": 30,  # negative particle
        "r_p": 20,  # positive particle
    }

    sim    = pybamm.Simulation(
        model, experiment = exp,
        parameter_values = para,
        solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,)       
    Sol_all_int.append(sim.solve())

Confirm: using interstitial-diffusion limited
Confirm: using interstitial-diffusion limited
<function dLJP_One_Specie_dce_Jung2023 at 0x000001D07EBD8550>
<function dLJP_Two_Species_dce_Jung2023 at 0x000001D07FDC3E50>


In [6]:
# sol - Dec=3e-10; single vs double 
Model_DFN_sol_lowDec  = pybamm.lithium_ion.DFN(options={
    "SEI":"solvent-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "single no consume wo refill",
    "electrolyte conductivity": "full"  ,}) 
Model_DD_sol_lowDec  = pybamm.lithium_ion.DFN(options={
    "SEI":"solvent-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "double no consume wo refill",
    "electrolyte conductivity": "sol full"  ,}) 
Model_All_sol_lowDec =[ Model_DFN_sol_lowDec ,   Model_DD_sol_lowDec ]
Str_model_sol_lowDec =[ 'Model_DFN_sol_lowDec', 'Model_DD_sol_lowDec' ]
# Para_All[0]: for single; Para_All[1]: for double
para=pybamm.ParameterValues("Li2023_ECdrag")
Para_All = []
for i in range(0,2):
    para=pybamm.ParameterValues("Li2023_ECdrag")
    para.update({"EC initial concentration in electrolyte [mol.m-3]":3500})
    para.update({"Typical EC concentration [mol.m-3]":3500})
    para.update({'EC diffusivity in SEI [m2.s-1]':5e-20})
    para.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_3E_10})
    para.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':3e-19})
    para.update({"EC Lithium ion cross diffusivity [m2.s-1]":Cross_diffusivity_1p5E_12})
    para.update({"Cation transference number":electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Con})
    Para_All.append(para)
Para_All[0].update({"Measured dLJP_dce":dLJP_One_Specie_dce_Jung2023})
# check 
print(Para_All[0]["Measured dLJP_dce"])
print(Para_All[1]["Measured dLJP_dce"])
Sol_all_sol_lowDec =[];
for model,para in zip(Model_All_sol_lowDec,Para_All):
    c_e = model.variables["Electrolyte concentration [mol.m-3]"]
    c_EC= model.variables["EC concentration [mol.m-3]"]
    T = model.variables["Cell temperature [K]"]
    D_e = para["Electrolyte diffusivity [m2.s-1]"]
    D_EC= para["EC diffusivity in electrolyte [m2.s-1]"]
    sigma_e = para["Electrolyte conductivity [S.m-1]"]
    Xi = para["EC transference number"]
    model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
    model.variables["EC diffusivity in electrolyte [m2.s-1]"] = D_EC(c_e,c_EC, T)
    model.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
    model.variables["EC transference number"] = Xi(c_e,c_EC, T)
    model.variables["c(EC) over c(Li+)"] = c_EC / c_e
    t_0plus = para["Cation transference number"]
    model.variables["Cation transference number"] = t_0plus(c_e,c_EC, T)

    var_pts = {
        "x_n": 10,  # negative electrode
        "x_s": 5,  # separator 
        "x_p": 10,  # positive electrode
        "r_n": 30,  # negative particle
        "r_p": 20,  # positive particle
    }

    sim    = pybamm.Simulation(
        model, experiment = exp,
        parameter_values = para,
        solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,)       
    Sol_all_sol_lowDec.append(sim.solve())

<function dLJP_One_Specie_dce_Jung2023 at 0x000001D07EBD8550>
<function dLJP_Two_Species_dce_Jung2023 at 0x000001D07FDC3E50>


In [7]:
# int - Dec=3e-10; single vs double 
Model_DFN_int_lowDec  = pybamm.lithium_ion.DFN(options={
    "SEI":"interstitial-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "single no consume wo refill",
    "electrolyte conductivity": "full"  ,}) 
Model_DD_int_lowDec  = pybamm.lithium_ion.DFN(options={
    "SEI":"interstitial-diffusion limited",             
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",   
    "solvent diffusion": "double no consume wo refill",
    "electrolyte conductivity": "sol full"  ,}) 
Model_All_int_lowDec =[ Model_DFN_int_lowDec ,   Model_DD_int_lowDec ]
Str_model_int_lowDec =[ 'Model_DFN_int_lowDec', 'Model_DD_int_lowDec' ]
# Para_All[0]: for single; Para_All[1]: for double
para=pybamm.ParameterValues("Li2023_ECdrag")
Para_All = []
for i in range(0,2):
    para=pybamm.ParameterValues("Li2023_ECdrag")
    para.update({"EC initial concentration in electrolyte [mol.m-3]":3500})
    para.update({"Typical EC concentration [mol.m-3]":3500})
    para.update({'EC diffusivity in SEI [m2.s-1]':5e-20})
    para.update({"EC diffusivity in electrolyte [m2.s-1]":EC_diffusivity_3E_10})
    para.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':3e-19})
    para.update({"EC Lithium ion cross diffusivity [m2.s-1]":Cross_diffusivity_1p5E_12})
    para.update({"Cation transference number":electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte conductivity [S.m-1]":electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Con})
    para.update({"Electrolyte diffusivity [m2.s-1]":electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Con})
    Para_All.append(para)
Para_All[0].update({"Measured dLJP_dce":dLJP_One_Specie_dce_Jung2023})
# check 
print(Para_All[0]["Measured dLJP_dce"])
print(Para_All[1]["Measured dLJP_dce"])
Sol_all_int_lowDec =[];
for model,para in zip(Model_All_int_lowDec,Para_All):
    c_e = model.variables["Electrolyte concentration [mol.m-3]"]
    c_EC= model.variables["EC concentration [mol.m-3]"]
    T = model.variables["Cell temperature [K]"]
    D_e = para["Electrolyte diffusivity [m2.s-1]"]
    D_EC= para["EC diffusivity in electrolyte [m2.s-1]"]
    sigma_e = para["Electrolyte conductivity [S.m-1]"]
    Xi = para["EC transference number"]
    model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
    model.variables["EC diffusivity in electrolyte [m2.s-1]"] = D_EC(c_e,c_EC, T)
    model.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
    model.variables["EC transference number"] = Xi(c_e,c_EC, T)
    model.variables["c(EC) over c(Li+)"] = c_EC / c_e
    t_0plus = para["Cation transference number"]
    model.variables["Cation transference number"] = t_0plus(c_e,c_EC, T)
    var_pts = {
        "x_n": 10,  # negative electrode
        "x_s": 5,  # separator 
        "x_p": 10,  # positive electrode
        "r_n": 30,  # negative particle
        "r_p": 20,  # positive particle
    }
    sim    = pybamm.Simulation(
        model, experiment = exp,
        parameter_values = para,
        solver = pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,)       
    Sol_all_int_lowDec.append(sim.solve())

Confirm: using interstitial-diffusion limited
Confirm: using interstitial-diffusion limited
<function dLJP_One_Specie_dce_Jung2023 at 0x000001D07EBD8550>
<function dLJP_Two_Species_dce_Jung2023 at 0x000001D07FDC3E50>


In [8]:
# all solutions to compare:
# Sol_all_sol          Str_model_sol
# Sol_all_sol_lowDec   Str_model_sol_lowDec
# Sol_all_int_lowDec;  Str_model_int_lowDec
# Sol_all_int          Str_model_int
model.variables.search("SEI")

EC source term (SEI)
Inner SEI concentration [mol.m-3]
Inner SEI interfacial current density
Inner SEI interfacial current density [A.m-2]
Inner SEI on cracks concentration [mol.m-3]
Inner SEI on cracks interfacial current density
Inner SEI on cracks interfacial current density [A.m-2]
Inner SEI on cracks thickness
Inner SEI on cracks thickness [m]
Inner SEI thickness
Inner SEI thickness [m]
Loss of capacity to SEI [A.h]
Loss of capacity to SEI on cracks [A.h]
Loss of lithium to SEI [mol]
Loss of lithium to SEI on cracks [mol]
Negative electrode SEI interfacial current density
Negative electrode SEI interfacial current density [A.m-2]
Negative electrode SEI on cracks interfacial current density
Negative electrode SEI on cracks interfacial current density [A.m-2]
Negative electrode SEI on cracks volumetric interfacial current density
Negative electrode SEI on cracks volumetric interfacial current density [A.m-3]
Negative electrode SEI volumetric interfacial current density
Negative elec

In [18]:
# compare sol - Dec
Str = [
    "Sol-Single-H_Dec","Sol-Double-H_Dec",
    "Sol-Single-L_Dec","Sol-Double-L_Dec",]
font = {'family' : 'DejaVu Sans','size'   : 12}
mpl.rc('font', **font)
var_elely = [
    "Terminal voltage [V]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    #"Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
    "Negative electrode SEI interfacial current density [A.m-2]",
    "X-averaged SEI interfacial current density [A.m-2]", # TO BE ADDED!
]
quick_plot = pybamm.QuickPlot(
    [
        Sol_all_sol[0],Sol_all_sol[1],
        Sol_all_sol_lowDec[0],Sol_all_sol_lowDec[1],
        ], var_elely,
    Str,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=0.4457476166152453, step=0.00445747616615245…

In [19]:
# compare int - Dec
Str = [
    "Int-Single-H_Dec","Int-Double-H_Dec",
    "Int-Single-L_Dec","Int-Double-L_Dec",]
font = {'family' : 'DejaVu Sans','size'   : 12}
mpl.rc('font', **font)
var_elely = [
    "Terminal voltage [V]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    #"Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
    "Negative electrode SEI interfacial current density [A.m-2]",
    "X-averaged SEI interfacial current density [A.m-2]",
]
quick_plot = pybamm.QuickPlot(
    [
        Sol_all_int[0],Sol_all_int[1],
        Sol_all_int_lowDec[0],Sol_all_int_lowDec[1],
        ], var_elely,
    Str,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=0.4474011447147022, step=0.00447401144714702…

In [ ]:
font = {'family' : 'DejaVu Sans','size'   : 17}
mpl.rc('font', **font)
output_variables3 = [
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Positive current collector potential [V]", 
    "Terminal voltage [V]", 
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol_all], output_variables3,Str_model,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

In [ ]:
Colors = ['b','k','r',];LS = ['-','--','-'];MS=['o','^','h'];Str=[ 'DFN',  'Single trans', 'Double trans' ]
fig, axs = plt.subplots(2,3, figsize=(10,6),tight_layout=True)
Keys_t_var = [
    "Battery voltage [V]",
    "X-averaged battery reaction overpotential [V]",
    "X-averaged battery concentration overpotential [V]",
    "X-averaged EC concentration overpotential [V]", # Mark Ruihe add
    "X-averaged battery electrolyte ohmic losses [V]",
    "X-averaged battery solid phase ohmic losses [V]",
]
Title_keys = [
    "Battery voltage",
    "Reaction overpotential",
    "c(Li+) overpotential",
    "c(EC) overpotential",
    "Elely ohmic losses",
    "Solid ohmic losses"
]
for k in range(0,3):
    step_no = 1
    time_1 = (
        Sol_all[k].cycles[0].steps[step_no]['Time [s]'].entries
        -Sol_all[k].cycles[0].steps[step_no]['Time [s]'].entries[0] )
    for i in range(0,6): # 0 1 2 3 4 5 --> [0,0] [0,1], [0,2], [1,0], [1,1], [1,2]   
        var =  Sol_all[k].cycles[0].steps[step_no][Keys_t_var[i]].entries
        if i < 3:
            m=0; n =i
        else:
            m=1; n=i-3
        axs[m,n].plot(time_1, var,color=Colors[k],linestyle=LS[k],label=Str[k]) # label="1st cycle"
        axs[m,n].set_title(Title_keys[i],fontdict={'family':'DejaVu Sans','size':fs-1})
        axs[m,n].set_ylabel("Potential [V]",   fontdict={'family':'DejaVu Sans','size':fs})
        axs[m,n].set_xlabel('Time [s]',   fontdict={'family':'DejaVu Sans','size':fs})
        #axs[m,n].set_ylim(-1.2,0.5)
        
        labels = axs[m,n].get_xticklabels() + axs[m,n].get_yticklabels(); 
        [label.set_fontname('DejaVu Sans') for label in labels]
        axs[0,0].legend(prop={'family':'DejaVu Sans','size':fs-5},loc='best',frameon=False)  
    #axs[0,0].set_ylim(2.49,4.2)